In [26]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [28]:
##create database
justwatch_db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="4156"
)

mycursor = justwatch_db.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS justwatch_movies_us_db")

In [29]:
mycursor.execute("use justwatch_movies_us_db")

In [30]:
mycursor.execute("""CREATE TABLE IF NOT EXISTS movie_links(title_link TEXT, is_scraped TINYINT)""")

In [31]:
##connect to db
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="justwatch_movies_us_db"
uname="root"
pwd="4156"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [32]:
base_url = "https://www.justwatch.com"

In [33]:
# Python3 Program to Create list 
# with integers within given range 
  
def get_search_links(year_from, year_to): 
    def createYears(year_from, year_to):
        return [item for item in range(year_from, year_to)]
      
    genres = ['act','cmy',  'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
    years = createYears(year_from, year_to)


    search_links = []
    for i in genres:
        for j in years:
            link = f"https://www.justwatch.com/us/movies?genres={i}&release_year_from={j}&release_year_until={j}"
            search_links.append(link)
            
    return search_links

In [35]:
def get_title_links(link):
    titleLinks = []
    print(link)
    chrome_options = Options()  
    chromedriver = webdriver.Chrome(ChromeDriverManager().install())
    with chromedriver as browser:
        browser.get(link)
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")
        SCROLL_PAUSE_TIME = 5
        while True:
                # Scroll down to bottom
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
                    title_link = title.find("a").get('href')
                    title_link = base_url+title_link
                    is_scraped = 0
                    if title_link not in titleLinks:
                        titleLinks.append(title_link)
                    
                    mycursor.execute("""INSERT INTO movie_links VALUES(%s, %s)""", (title_link, is_scraped))
                    justwatch_db.commit()
                    
        
        return titleLinks
                    
                        
                        

In [15]:
search_urls = get_search_links(2020, 2022)
for link in search_urls:
    get_title_links(link)

https://www.justwatch.com/us/movies?genres=act&release_year_from=2020&release_year_until=2020


C:\Users\user\AppData\Local\Temp\ipykernel_5564\3104721579.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chromedriver = webdriver.Chrome(ChromeDriverManager().install())


https://www.justwatch.com/us/movies?genres=act&release_year_from=2021&release_year_until=2021
https://www.justwatch.com/us/movies?genres=cmy&release_year_from=2020&release_year_until=2020
https://www.justwatch.com/us/movies?genres=cmy&release_year_from=2021&release_year_until=2021
https://www.justwatch.com/us/movies?genres=doc&release_year_from=2020&release_year_until=2020
https://www.justwatch.com/us/movies?genres=doc&release_year_from=2021&release_year_until=2021
https://www.justwatch.com/us/movies?genres=fnt&release_year_from=2020&release_year_until=2020
https://www.justwatch.com/us/movies?genres=fnt&release_year_from=2021&release_year_until=2021
https://www.justwatch.com/us/movies?genres=hrr&release_year_from=2020&release_year_until=2020
https://www.justwatch.com/us/movies?genres=hrr&release_year_from=2021&release_year_until=2021
https://www.justwatch.com/us/movies?genres=msc&release_year_from=2020&release_year_until=2020
https://www.justwatch.com/us/movies?genres=msc&release_year_

In [36]:
df_links = pd.read_sql('SELECT * FROM movie_links WHERE is_scraped = 0', con=justwatch_db)
title_urls = []
for i in df_links['title_link']:
    title_urls.append(i)

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [37]:
len(title_urls)

22134

In [18]:
def get_detail(soup_object):
    labels  = []
    values = []
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

In [19]:
def get_stream_options(soup_object):
        stream_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)
        return stream_list


def get_rent_options(soup_object):
        rent_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list
    
def get_buy_options(soup_object):
        buy_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list

In [20]:
def get_best(driver):
            driver.implicitly_wait(0.5)
            '''
            best = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[3]/div/div[2]/div/div[1]/div[1]/div/div[1]'))
            )
            best.click()
            
            time.sleep(5)
            '''
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_best = get_stream_options(hun)
            buy_best = get_buy_options(hun)
            rent_best = get_rent_options(hun)
            
            options = {"stream_best": stream_best,
                    "rent_best":rent_best,
                    "buy_best":buy_best}
            
            return options
        
def get_free(driver, bts):
            time.sleep(5)
            bts[1].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_free = get_stream_options(hun)
            buy_free = get_buy_options(hun)
            rent_free = get_rent_options(hun)
            
            
            options = {"stream_free": stream_free,
                    "rent_free":rent_free,
                    "buy_free":buy_free}
        
            return options
        
def get_sd(driver, bts):
            driver.implicitly_wait(0.5)
            bts[2].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_sd = get_stream_options(hun)
            buy_sd = get_buy_options(hun)
            rent_sd = get_rent_options(hun)
            
            options = {"stream_sd": stream_sd,
                    "rent_sd":rent_sd,
                    "buy_sd":buy_sd}
        
            return options
        
def get_hd(driver, bts):
        driver.implicitly_wait(0.5)
            
        bts[3].click()        
        time.sleep(5)
            
        page_source =driver.page_source
        
        hun=BeautifulSoup(page_source,'html.parser')
        stream_hd = get_stream_options(hun)
        buy_hd = get_buy_options(hun)
        rent_hd = get_rent_options(hun)
            
        options = {"stream_hd": stream_hd,
                    "rent_hd":rent_hd,
                    "buy_hd":buy_hd}
        
        return options
        
      
def get_4k(driver, bts):   
        driver.implicitly_wait(0.5)
        bts[4].click()
            
        time.sleep(5)
            
        page_source =  driver.page_source
        hun=BeautifulSoup(page_source,'html.parser')
        stream_4k = get_stream_options(hun)
        buy_4k = get_buy_options(hun)
        rent_4k = get_rent_options(hun)       
            
        options = {"stream_4k": stream_4k,
                    "rent_4k":rent_4k,
                    "buy_4k":buy_4k}
        
        return options

In [21]:
def get_data(link):
            driver = webdriver.Chrome(ChromeDriverManager().install())
            with driver as driver:
                driver.get(link)
                banner = WebDriverWait(driver=driver, timeout=5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
                    )
                
                banner.click()
                
                time.sleep(1)
                bts = driver.find_elements(By.CLASS_NAME,'jw-chip-button')
                
                time.sleep(1)
                try:
                    imdb_link = WebDriverWait(driver=driver, timeout=5).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[1]/div[2]/div/div[2]/a'))
                    )
               
                    imdb_link = imdb_link.get_attribute('href')
                except:
                    imdb_link = None
                
                page_source =  driver.page_source
                
                hun=BeautifulSoup(page_source,'html.parser')
                
                
                try:
                    detail = get_detail(hun)
                except:
                    detail = None
                    
                try:
                    title_name = hun.find("div", {"class":"title-block"})
                    title_name = title_name.find("h1").text.replace('\n',"")
                except:
                    title_name = None
                    
                try:
                    year = hun.find("span", {"class":"text-muted"}).text.replace('\n',"")
                except:
                    year = None
                
                try:
                    prices_best = get_best(driver)
                except:
                    prices_best = None
                
                try:
                    prices_free = get_free(driver,bts)
                except:
                    prices_free = None
                    
                try:
                    prices_sd = get_sd(driver, bts)
                except:
                    prices_sd = None
                    
                try:
                    prices_hd = get_hd(driver, bts)
                except:
                    prices_hd = None

                try:   
                    prices_4k = get_4k(driver, bts)
                except:
                    prices_4k = None
        
        
                title_url = link
                
                try:
                    now = datetime.now()
                    date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
                except:
                    date_scraped = None
                
                tv = {"title":title_name, "year":year, "detail":detail, "4k prices":prices_4k, "hd prices":prices_hd, "sd prices":prices_sd, "free":prices_free, "best prices":prices_best,\
                    "imdb_link":imdb_link,"tite url": title_url,
                      "date_scraped":date_scraped}

        
            return  tv
        
        
        

In [38]:
for link in title_urls[2001:2020]:
    data = []
    data.append(get_data(link))
    df = pd.DataFrame.from_dict(data).astype("string")
    df.to_sql('data_table', engine, if_exists='append', index=False)
    sql = "UPDATE movie_links SET is_scraped = 1 WHERE title_link = (%s)"
    titlelink = (link,)
    mycursor.execute(sql, titlelink)
    justwatch_db.commit()

C:\Users\user\AppData\Local\Temp\ipykernel_5564\2154764294.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
df = pd.read_sql('SELECT * FROM data_table', con=justwatch_db)

In [ ]:
df

,title,year,detail,4k prices,hd prices,sd prices,free,best prices,imdb_link,tite url
0,Laal Zameen,(2022),"{'Rating': ' 4% ', 'Genres': 'Action & Adventu...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [], 'rent_hd': [{'Amazon Video':...","{'stream_sd': [], 'rent_sd': [{'Amazon Video':...","{'stream_free': [], 'rent_free': [], 'buy_free...","{'stream_best': [], 'rent_best': [{'Amazon Vid...",None,https://www.justwatch.com/us/movie/laal-zameen
1,The Mannequin,(2022),"{'Rating': ' 4% ', 'Genres': 'Science-Fiction ...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [], 'rent_hd': [{'Amazon Video':...","{'stream_sd': [{'Tubi TV': 'Ads '}], 'rent_sd'...","{'stream_free': [{'Tubi TV': 'Ads '}], 'rent_f...","{'stream_best': [{'Tubi TV': 'Ads '}], 'rent_b...",None,https://www.justwatch.com/us/movie/the-mannequin


In [ ]:
k_df = df['4k prices'].apply(pd.Series)
h_df = df['hd prices'].apply(pd.Series)
s_df = df['sd prices'].apply(pd.Series)
f_df = df['free'].apply(pd.Series)
b_df = df['best prices'].apply(pd.Series)
d_df = df['detail'].apply(pd.Series)
d_df

,Rating,Genres,Runtime,Director
0,4%,"Action & Adventure , Drama",2h 32min,Vivek Shrivastava
1,4%,"Science-Fiction , Animation , Action & Adventure",1h 4min,Jourin Hannah


In [ ]:
df = pd.concat([df, k_df.reindex(df.index)], axis=1)
df = pd.concat([df, h_df.reindex(df.index)], axis=1)
df = pd.concat([df, s_df.reindex(df.index)], axis=1)
df = pd.concat([df, f_df.reindex(df.index)], axis=1)
df = pd.concat([df, b_df.reindex(df.index)], axis=1)
df = pd.concat([df, d_df.reindex(df.index)], axis=1)

df_c = df.drop(['4k prices', 'hd prices', 'sd prices', 'free', 'best prices', 'detail'], axis=1)

In [ ]:
df_c

,title,year,imdb_link,tite url,stream_4k,rent_4k,buy_4k,stream_hd,rent_hd,buy_hd,...,stream_free,rent_free,buy_free,stream_best,rent_best,buy_best,Rating,Genres,Runtime,Director
0,Laal Zameen,(2022),None,https://www.justwatch.com/us/movie/laal-zameen,[],[],[],[],[{'Amazon Video': '$2.99 HD '}],[{'Amazon Video': '$9.99 HD '}],...,[],[],[],[],[{'Amazon Video': '$1.99 '}],[{'Amazon Video': '$7.99 '}],4%,"Action & Adventure , Drama",2h 32min,Vivek Shrivastava
1,The Mannequin,(2022),None,https://www.justwatch.com/us/movie/the-mannequin,[],[],[],[],[{'Amazon Video': '$2.99 HD '}],[{'Amazon Video': '$9.99 HD '}],...,[{'Tubi TV': 'Ads '}],[],[],[{'Tubi TV': 'Ads '}],[{'Amazon Video': '$1.99 '}],[{'Amazon Video': '$7.99 '}],4%,"Science-Fiction , Animation , Action & Adventure",1h 4min,Jourin Hannah


In [ ]:
# Convert dataframe to sql table                                   
df_c.to_sql('justwatch_data_movies_us_clean',  engine, if_exists='replace', index=False)

In [ ]:
df.to_csv("movies.csv")